# Primeiros passos com o TensorFlow

Para iniciar a nossa prática usando o TensorFlow, construiremos uma rede neural simples *Single-Layer Perceptron*, com apenas uma camada escondida, para classificar dígitos de 0 à 9 escritos à mão.

Comece importando o tensorflow no seu ambiente, como mostrado abaixo.

In [ ]:
import tensorflow as tf 

## Dataset

Precisamos organizar um conjunto de imagens de dígitos escritos à mão separados por classes para que possamos treinar a nossa rede neural. Geralmente, essa é parte mais trabalhosa de experimentos com Inteligência Artificial. 

## A boa notícia 

Felizmente, o TensorFlow já possui um dataset com as características que precisamos definido internamente. Então, só precisamos importá-lo para o nosso ambiente.

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

# Faz o download das imagens e das suas labels correspondentes como one-hot vectors 
mnist_dataset = input_data.read_data_sets("mnist_dataset/", one_hot = True) 

## Informações do dataset 

Se você for até o diretório onde esse notebook está, verá que uma nova pasta com o nome **mnist_dataset** foi criada. Entrando na pasta, você vai encontrar 4 arquivos, que contém as imagens que serão utilizadas para treino e validação e suas classes correspondentes.

Vamos verificar a quantidade de exemplos que serão utilizados para treino e para teste. 

In [ ]:
print("Informações do dataset")
print("Dataset pra treino:\t\t{}".format(len(mnist_dataset.train.labels)))
print("Dataset pra teste:\t\t{}".format(len(mnist_dataset.test.labels)))

## Visualizando os dados

Para visualizar as imagens que compõem o dataset, utilize  a função utilitária fornecida logo abaixo para plotar 9 exemplos de imagens do dataset.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

# Função utilitária para plotagem das imagens contidas no dataset.
def plot_images(images, target_class, image_shape, predicted_class = None):
    
    # Certifica-se de que o número de imagens e classes são ambos iguais a 9.
    assert len(images) == len(target_class) == 9

    # Cria uma figura com 3 subplots por linha e 3 subplots por coluna.
    figure, axes = plt.subplots(3,3)
    
    # Ajusta o espaçamento entre um subplot e outro.
    figure.subplots_adjust(hspace = 0.3, wspace = 0.3)
    
    # Itera em cada um dos subplots, preenchendo-os com as imagens passadas como argumento.
    for i, ax in enumerate(axes.flat):
        
        # Faz o reajuste da imagem para 2 dimensões e preenche um subplot.
        ax.imshow(images[i].reshape(image_shape), cmap = 'binary') 
        
        # Se as classes previstas não forem passadas para função, então exibimos apenas a classe original da imagem.
        if predicted_class is None:
            x_label = "Target: {0}".format(target_class[i])
        # Se as classes previstas forem passadas para função, exibiremos o resultado da previsão e a classe original.
        else:
            x_label = "Target: {0}, Predicted: {1}".format(target_class[i], predicted_class[i])
        # Seta a legenda da imagem
        ax.set_xlabel(x_label)
        
        # Seta o eixo x do subplot como sendo vazio
        ax.set_xticks([]) 
        
        #Seta o eixo y do subplot como sendo vazio
        ax.set_yticks([]) 
    
    # Plota a figura 
    plt.show()

In [ ]:
import numpy as np

# Coleta as nove primeiras imagens do dataset de teste
images = mnist_dataset.test.images[10:19]

# Converte as classes do formato one-hot para o formato  numérico
mnist_dataset.test.cls =  np.array([label.argmax() for label in mnist_dataset.test.labels])
target_class = mnist_dataset.test.cls[10:19]

In [ ]:
plot_images(images = images, target_class = target_class, image_shape = (28,28))

## Estruturação dos scripts

Os scripts feitos em TensorFlow são estruturados em dois blocos principais, que são:

1. Grafos de fluxo de dados;
2. Sessões.

Os grafos de fluxo de dados definem todas todas as variáveis e operações do modelo que desejamos executar, neste caso uma rede neural. Dependendo de quão grande seja o seu projeto, você pode definir vários grafos diferentes. Nesse tutorial um grafo já será suficiente para o que desejamos fazer.

Então, vamos criar o nosso grafo e definir toda a estrutura da nossa rede neural, como no snippet logo abaixo.

In [ ]:
# Define o tamanho das imagens(quadradas) que serão aceitas pelo modelo
image_size = 28

# Define o número de classes que serão trabalhadas no modelo
number_of_classes = 10

# Cria grafo de fluxo de dados
graph = tf.Graph()
    
# Define os componentes e as operações que serão realizadas no modelo
with graph.as_default():
    
    with tf.name_scope("Input-Layer") as scope:
        
        # Cria um placeholder para o tensor que irá guardar as imagens de entrada (achatadas em uma única dimensão)
        input_images = tf.placeholder(tf.float32, [None, image_size * image_size])

        # Cria um place holder para guardar a classe de cada uma das imagens na forma de one-hot vectors
        image_labels = tf.placeholder(tf.float32, [None, number_of_classes])

        # Cria um placeholder para guardar o valor numérico da classe de cada imagem 
        image_classes = tf.placeholder(tf.int64, [None])
        
    with tf.name_scope("Output-Layer") as scope:
        
        # Tensor de pesos da camada escondida 
        weights = tf.Variable(tf.zeros([image_size * image_size, number_of_classes]))

        # Tensor que guarda os valores de bias 
        biases = tf.Variable(tf.zeros([number_of_classes]))

        # Faz o produto matricial da entrada com os pesos da rede e soma com o bias
        logits = tf.matmul(input_images, weights) + biases

        # Aplica a função de ativação softmax na saída 
        prediction = tf.nn.softmax(logits)

        # Pega o índice da posição do tensor com maior argumento 
        predicted_classes = tf.argmax(prediction, dimension = 1)
    
    with tf.name_scope("Optimization") as scope:

        # Calcula o cross-entropy (função de custo)
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = image_labels)

        # Calcula o "custo" da função que desejamos otimizar
        loss = tf.reduce_mean(cross_entropy)

        # Calcula o gradiente da função 
        optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5).minimize(loss)

        # Compara as classes que foram previstas pela rede e as classes reais de cada imagem
        correct_prediction = tf.equal(predicted_classes, image_classes)

        # Calcula a acurária da rede
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # create a summary for our cost and accuracy
    tf.summary.scalar("cost", cross_entropy)
    tf.summary.scalar("accuracy", accuracy)

    # merge all summaries into a single "operation" which we can execute in a session 
    summary_op = tf.summary.merge_all()

## Sessão

As sessões funcionam como conexões entre os grafos definidos nos scripts e os dispositivos em nossas maquinas ou dispositivos remotos. As operações que definimos no nosso grafo só serão realmente realizadas depois que uma sessão for inicializada para aquele grafo. 

Vimos anteriormente que nós podemos definir vários grafos dentro de um mesmo projeto, porém **uma sessão executa apenas um grafo**. Então, se você tiver que lidar como vários grafos ao mesmo tempo você terá criar uma sessão pra cada grafo separadamente, ou criar um novo grafo para incluir os seuss grafos como subgrafos dele.

In [ ]:
with tf.Session(graph = graph) as session:
    # Inicializa todas as variáveis no modelo
    tf.global_variables_initializer().run()
    
    # Define a quantidade de imagens por batch
    mini_batch_size = 256
    
    # Número de backpropagations que serão utilizados para treinar o modelo
    number_of_iterations = 50
    
    logs_path = "summary/logs"
    
    # Define a quantidade de imagens por batch
    mini_batch_size = 256
    
    writer = tf.summary.FileWriter(logs_path, graph= graph)
    
    # Inicia o treinamento
    for i in range(number_of_iterations):
        
        # Gera um batch de imagens para alimentar a rede
        input_images_batch, image_labels_batch = mnist_dataset.train.next_batch(mini_batch_size)
       
        # Dicionário que os dados de treino da rede
        feed_dict_train = {input_images: input_images_batch,
                           image_labels: image_labels_batch}
        
        # Executa o otimizador do modelo
        _ = session.run(optimizer, feed_dict = feed_dict_train)
        
        # write log
        #writer.add_summary(summary, i)
    
    # Dicionário que guarda os dados de teste da rede
    feed_dict_test = {input_images: mnist_dataset.test.images,
                      image_labels: mnist_dataset.test.labels,
                      image_classes: mnist_dataset.test.cls}
    # Acurácia do modelo
    model_accuracy = session.run(accuracy, feed_dict = feed_dict_test)
    print("Acurácia no dataset de teste: {0:.1%}".format(model_accuracy))

## Visualizando o grafo 

Para visualizar o grafo do modelo que acabamos de definir, vá até a pasta /summary/logs e copie o caminho completo até um dos arquivos contidos na pasta. Em seguida, abre o terminal e digite o seguinte comando:

**tensorboard --logdir=caminho_copiado --port 6006**

## Visualizando a convergência do modelo

Uma boa maneira de analisar visualmente a convergência de um modelo que classifica imagens é através da matriz de confusão. Vamos utilizar a função fornecida logo abaixo para analisar como as imagens estavam sendo classificadas antes e depois do treinamento.

In [ ]:
from sklearn.metrics import confusion_matrix 

def plot_confusion_matrix(feed_dict, target_class, session):
    
    # Faz a predição das classes de cada uma das imagens
    pred_class = session.run(predicted_classes, feed_dict = feed_dict)
    
    # Cria uma matriz de confusão das classes preditas pelas classes reais
    model_confusion_matrix = confusion_matrix(y_true = target_class,
                                    y_pred = pred_class)
    
    # Plota a matriz
    plt.imshow(model_confusion_matrix, interpolation = 'nearest', cmap = plt.cm.ocean)
    
    plt.tight_layout()
    plt.colorbar()
    tick_marks = np.arange(number_of_classes)
    plt.xticks(tick_marks, range(number_of_classes))
    plt.yticks(tick_marks, range(number_of_classes))
    plt.xlabel('Predicted')
    plt.ylabel('Target')

    plt.show()

## Antes do treinamento

Primeiramente, vamos verificar como o nosso modelo está classificado as imagens antes do treinamento.

In [ ]:
# Dicionário que guarda os dados de teste da rede
feed_dict_test = {input_images: mnist_dataset.test.images,
                  image_labels: mnist_dataset.test.labels,
                  image_classes: mnist_dataset.test.cls}

# Cria uma sessão
with tf.Session(graph = graph) as session:
    #Inicializa todas as variáveis do modelo
    tf.global_variables_initializer().run()
    
    # Calcula a acurácia do modelo antes do treinamento
    model_accuracy = session.run(accuracy, feed_dict = feed_dict_test)
    
    # Imprime a acurácia do modelo
    print("Acurácia no dataset de teste: {0:.1%}".format(model_accuracy))
    
    # Plota a matriz de confusão do modelo antes de ser treinado
    plot_confusion_matrix(feed_dict_test, mnist_dataset.test.cls, session)

## Após o treinamento

Quando plotamos a matriz de confusão anteriormente verificamos que a maioria das imagens estavam sendo classificadas de forma errada, mas como será que ela vai ficar após o treinamento?

Vamos ver agora!

In [ ]:
with tf.Session(graph = graph) as session:
    # Inicializa todas as variáveis no modelo
    tf.global_variables_initializer().run()
    
    # Define a quantidade de imagens por batch
    mini_batch_size = 256
    
    # Número de backpropagations que serão utilizados para treinar o modelo
    number_of_iterations = 1000
    
    # Inicia o treinamento
    for i in range(number_of_iterations):
        
        # Gera um batch de imagens para alimentar a rede
        input_images_batch, image_labels_batch = mnist_dataset.train.next_batch(mini_batch_size)
        
        # Dicionário que os dados de treino da rede
        feed_dict_train = {input_images: input_images_batch,
                           image_labels: image_labels_batch}
        
        # Executa o otimizador do modelo
        session.run(optimizer, feed_dict = feed_dict_train)
   
    # Calcula a acurácia do modelo após o treinamento 
    model_accuracy = session.run(accuracy, feed_dict = feed_dict_test)
    
    # Imprime a acurácia do modelo
    print("Acurácia no dataset de teste: {0:.1%}".format(model_accuracy))
    
    # Plota a matriz de confusão após o treinamento da rede
    plot_confusion_matrix(feed_dict_test, mnist_dataset.test.cls, session)

# Nosso modelo aprendeu direitinho!

Através da matriz de confusão podemos ver claramente como o nosso modelo está interpretando cada imagem e quais as classes de imagens que ele ainda sente um pouco de dificuldade para classificar.

### Mas será que conseguimos visualizar o que a nossa rede neural está aprendendo de fato?

A resposta é **sim**!

Vimos durante o minicurso que a magia por trás de uma rede neural está em suas matrizes de pesos. Os pesos são o que, de fato, caracterizam o que a rede aprendeu. Então, **o que aconteceria se a gente tentasse plotar a nossa matriz de pesos?**

In [ ]:
def plot_weights(session, image_shape):
    
    # Retorna o valor da matriz de pesos do modelo
    model_weights = session.run(weights)
    
    # Guarda o menor valor de peso da matriz
    min_weight = np.min(model_weights)
    
    # Guarda o maior valor de peso da matriz
    max_weight = np.max(model_weights)
    
    # Cria uma figura com 12 subplots
    fig, axes = plt.subplots(3,4)
    
    # Ajusta o espaçamento entre cada um dos subplots
    fig.subplots_adjust(hspace = 0.3, wspace = 0.3)
    
    # Itera em cada um dos subplots criados
    for i, ax in enumerate(axes.flat):
        # Preenche apenas 10 subplots, pois temos apenas 10 classes
        if i < 10:
            # Faz o reshape de uma coluna da matriz de pesos para 2 dimensões
            image = model_weights[:, i].reshape(image_shape)
            
            # Insere uma legenda para o subplot
            ax.set_xlabel("Weights: {0}".format(i))
            
            # Plota a imagem da coluna, representado os menos expressivos em azul e os mais expressivos em vermelho
            ax.imshow(image, vmin= min_weight, vmax = max_weight, cmap = 'seismic')
        
        # Seta o eixo x como sendo vazio
        ax.set_xticks([])
        
        # Seta o eixo y como sendo vazio
        ax.set_yticks([])
    
    # Plota a figura
    plt.show()

Tente usar a função fornecidade acima para visualizar os pesos da rede antes e depois do treinamento, de modo similar ao que você fez com a matriz de confusão.

In [ ]:
with tf.Session(graph = graph) as session:
    # Inicializa todas as variáveis do modelo
    tf.global_variables_initializer().run()
    
    # Calcula a acurácia 
    model_accuracy = session.run(accuracy, feed_dict = feed_dict_test)
    
    # Imprime a acurácia do modelo
    print("Acurácia no dataset de teste: {0:.1%}".format(model_accuracy))
    
    # Plota os pesos da rede
    plot_weights(session, (28,28))

# Não consigo visualizar nada :(

Tudo bem! Esse é o resultado esperado. A resposta pra todas essas imagens escuras que estamos vendo está no fato de que a nossa matriz de pesos foi completamente inicializada com zeros, como você pode ver se voltar ao snippet onde definimos o nosso grafo. Por isso, todos os subplots estão preenchidos com a cor azul escuro, pois essa cor representa o valor do menor peso presente na matriz, que nesse caso é zero.

## O que será que vai acontecer quando treinarmos a rede?

In [ ]:
with tf.Session(graph = graph) as session:
    # Inicializa todas as variáveis no modelo
    tf.global_variables_initializer().run()
    
    # Define a quantidade de imagens por batch
    mini_batch_size = 256
    
    # Número de backpropagations que serão utilizados para treinar o modelo
    number_of_iterations = 10
    
    # Inicia o treinamento
    for i in range(number_of_iterations):
        # Gera um batch de imagens para alimentar a rede
        input_images_batch, image_labels_batch = mnist_dataset.train.next_batch(mini_batch_size)
        
        # Dicionário que os dados de treino da rede
        feed_dict_train = {input_images: input_images_batch,
                           image_labels: image_labels_batch}
        
        # Executa o otimizador
        session.run(optimizer, feed_dict = feed_dict_train)
   
    # Calcula a acurácia do modelo
    model_accuracy = session.run(accuracy, feed_dict = feed_dict_test)
    
    # Imprime acurácia do modelo
    print("Acurácia no dataset de teste: {0:.1%}".format(model_accuracy))
    
    # Plota os pesos da rede
    plot_weights(session, (28,28))